In [1]:
! pip install scipy tenacity tiktoken termcolor openai requests

In [38]:
from openai import OpenAI
import httpx 

client = OpenAI(
    base_url=""
)

GPT_MODEL= "gpt-4o"

In [39]:
# 定义一个名为functions的列表，其中包含两个字典，这两个字典分别定义了两个功能的相关参数

# 第一个字典定义了一个名为"get_current_weather"的功能
functions = [
    {
        "name": "get_current_weather",  # 功能的名称
        "description": "Get the current weather",  # 功能的描述
        "parameters": {  # 定义该功能需要的参数
            "type": "object",
            "properties": {  # 参数的属性
                "location": {  # 地点参数
                    "type": "string",  # 参数类型为字符串
                    "description": "The city and state, e.g. San Francisco, CA",  # 参数的描述
                },
                "format": {  # 温度单位参数
                    "type": "string",  # 参数类型为字符串
                    "enum": ["celsius", "fahrenheit"],  # 参数的取值范围
                    "description": "The temperature unit to use. Infer this from the users location.",  # 参数的描述
                },
            },
            "required": ["location", "format"],  # 该功能需要的必要参数
        },
    },
    # 第二个字典定义了一个名为"get_n_day_weather_forecast"的功能
    {
        "name": "get_n_day_weather_forecast",  # 功能的名称
        "description": "Get an N-day weather forecast",  # 功能的描述
        "parameters": {  # 定义该功能需要的参数
            "type": "object",
            "properties": {  # 参数的属性
                "location": {  # 地点参数
                    "type": "string",  # 参数类型为字符串
                    "description": "The city and state, e.g. San Francisco, CA",  # 参数的描述
                },
                "format": {  # 温度单位参数
                    "type": "string",  # 参数类型为字符串
                    "enum": ["celsius", "fahrenheit"],  # 参数的取值范围
                    "description": "The temperature unit to use. Infer this from the users location.",  # 参数的描述
                },
                "num_days": {  # 预测天数参数
                    "type": "integer",  # 参数类型为整数
                    "description": "The number of days to forecast",  # 参数的描述
                }
            },
            "required": ["location", "format", "num_days"]  # 该功能需要的必要参数
        },
    },
]

In [22]:
messages = []

messages.append({
    "role": "system",
    "content": "不要对函数的值做任何假设，所有函数参数均使用英语，如果对于用户的输入有任何不清楚的地方，请提问用户。"
})

messages.append({
    "role": "user",
    "content": "今天上海天气怎么样"
})

resp = client.chat.completions.create(
    model=GPT_MODEL,
    temperature=0,
    messages=messages,
    functions=functions
)

print( resp.choices[0].message.function_call if resp.choices[0].message.function_call else resp.choices[0].message.content)

FunctionCall(arguments='{"location":"Shanghai, China","format":"celsius"}', name='get_current_weather')


In [23]:
messages.append({
    "role": "user",
    "content": "未来三天天气怎么样？"
})
resp = client.chat.completions.create(
    model=GPT_MODEL,
    temperature=0,
    messages=messages,
    functions=functions
)
print( resp.choices[0].message.function_call if resp.choices[0].message.function_call else resp.choices[0].message.content)


FunctionCall(arguments='{"location":"Shanghai, China","format":"celsius","num_days":3}', name='get_n_day_weather_forecast')


In [31]:
resp = client.chat.completions.create(
    model=GPT_MODEL,
    temperature=0,
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"}
)
print( resp.choices[0].message.function_call if resp.choices[0].message.function_call else resp.choices[0].message.content)


FunctionCall(arguments='{"location":"Shanghai, China","format":"celsius"}', name='get_current_weather')


In [40]:
import sqlite3

conn = sqlite3.connect("data/chinook.db")


In [58]:
def get_table_names(conn):
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    return [row[0] for row in cursor.fetchall()]

def get_table_schema(conn, table_name):
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name})")
    return cursor.fetchall()

def get_database_info(conn):
    """返回一个字典列表，每个字典包含一个表的名字和列信息"""
    table_dicts = []  # 创建一个空的字典列表
    # 遍历数据库中的所有表
    for table_name in get_table_names(conn):
        columns_names = get_table_schema(conn, table_name)  # 获取当前表的所有列名
        # 将表名和列名信息作为一个字典添加到列表中
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts  # 返回字典列表

get_database_info(conn)
get_table_schema(conn, "albums")
get_database_info(conn)


[{'table_name': 'albums',
  'column_names': [(0, 'AlbumId', 'INTEGER', 1, None, 1),
   (1, 'Title', 'NVARCHAR(160)', 1, None, 0),
   (2, 'ArtistId', 'INTEGER', 1, None, 0)]},
 {'table_name': 'sqlite_sequence',
  'column_names': [(0, 'name', '', 0, None, 0), (1, 'seq', '', 0, None, 0)]},
 {'table_name': 'artists',
  'column_names': [(0, 'ArtistId', 'INTEGER', 1, None, 1),
   (1, 'Name', 'NVARCHAR(120)', 0, None, 0)]},
 {'table_name': 'customers',
  'column_names': [(0, 'CustomerId', 'INTEGER', 1, None, 1),
   (1, 'FirstName', 'NVARCHAR(40)', 1, None, 0),
   (2, 'LastName', 'NVARCHAR(20)', 1, None, 0),
   (3, 'Company', 'NVARCHAR(80)', 0, None, 0),
   (4, 'Address', 'NVARCHAR(70)', 0, None, 0),
   (5, 'City', 'NVARCHAR(40)', 0, None, 0),
   (6, 'State', 'NVARCHAR(40)', 0, None, 0),
   (7, 'Country', 'NVARCHAR(40)', 0, None, 0),
   (8, 'PostalCode', 'NVARCHAR(10)', 0, None, 0),
   (9, 'Phone', 'NVARCHAR(24)', 0, None, 0),
   (10, 'Fax', 'NVARCHAR(24)', 0, None, 0),
   (11, 'Email', 'NVARC

In [73]:
import json


def ask_database(conn, query):
    try:
        res = str(conn.execute(query).fetchall())
    except Exception as e:
        res = f"Query failed with error: {e}"
    return res 

def execute_function_call(message):
    if message.function_call.name == "ask_database":
        sql = json.loads(message.function_call.arguments)["query"] 
        res = ask_database(conn, sql)
    else:
        res = f"Error: function {message.function.name} does not exist"
    return res 

db_schema_dic = get_database_info(conn)

In [48]:
functions = [
    {
        "name": "ask_database",
        "description": "Use this function to answer user questions about music. Output should be a fully formed SQL query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            SQL query extracting info to answer the user's question.
                            SQL should be written using this database schema:
                            {db_schema_dic}
                            The query should be returned in plain text, not in JSON.
                            """,
                }
            },
            "required": ["query"],
        },
    }
]

In [76]:
messages = []
# 向消息列表中添加一个系统角色的消息，内容是 "Answer user questions by generating SQL queries against the Chinook Music Database."
messages.append({"role": "system", "content": "Answer user questions by generating SQL queries against the Chinook Music Database."})

# 向消息列表中添加一个用户角色的消息，内容是 "Hi, who are the top 5 artists by number of tracks?"
messages.append({"role": "user", "content": "Hi, who are the top 5 artists by number of tracks?"})

resp = client.chat.completions.create(
    model=GPT_MODEL,
    temperature=0,
    messages=messages,
    functions=functions
)

assistant_message = resp.choices[0].message


if resp.choices[0].finish_reason == "function_call":
     # 使用 execute_function_call 函数执行功能调用，并获取结果
    results = execute_function_call(assistant_message)
    # 将功能的结果作为一个功能角色的消息添加到消息列表中
    messages.append({"role": "function", "name": assistant_message.function_call.name, "content": results})

messages

[{'role': 'system',
  'content': 'Answer user questions by generating SQL queries against the Chinook Music Database.'},
 {'role': 'user',
  'content': 'Hi, who are the top 5 artists by number of tracks?'},
 {'role': 'function',
  'name': 'ask_database',
  'content': "[('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]"}]

In [79]:
messages.append({"role": "user", "content": "哪一个专辑最受欢迎，告诉我最受欢迎那一个"})

resp = client.chat.completions.create(
    model=GPT_MODEL,
    temperature=0,
    messages=messages,
    functions=functions
)

assistant_message = resp.choices[0].message

if resp.choices[0].finish_reason == "function_call":
    results = execute_function_call(assistant_message)
    # 将功能的结果作为一个功能角色的消息添加到消息列表中
    messages.append({"role": "function", "name": assistant_message.function_call.name, "content": results})

messages

[{'role': 'system',
  'content': 'Answer user questions by generating SQL queries against the Chinook Music Database.'},
 {'role': 'user',
  'content': 'Hi, who are the top 5 artists by number of tracks?'},
 {'role': 'function',
  'name': 'ask_database',
  'content': "[('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]"},
 {'role': 'user', 'content': '哪一个专辑最有名？'},
 {'role': 'function',
  'name': 'ask_database',
  'content': "[('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]"},
 {'role': 'user', 'content': '哪一个专辑最受欢迎，告诉我最受欢迎那一个'},
 {'role': 'function',
  'name': 'ask_database',
  'content': "[('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]"},
 {'role': 'user', 'content': '哪一个专辑最受欢迎，告诉我最受欢迎那一个'},
 {'role': 'function',
  'name': 'ask_database',
  'content': "[('Minha Historia', 27)]"}]